In [1]:
%load_ext autoreload
%autoreload 2

In [61]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [60]:
import numpy as np
import pandas as pd
from kcsd import KCSD1D

In [226]:
from ecephys.sglx_utils import load_timeseries
from ecephys.data import channel_groups, paths
from ecephys.plot import lfp_explorer, colormesh_timeseries_explorer
from ecephys.signal.utils import mean_subtract, median_subtract
from ecephys.signal.csd import get_kcsd

In [242]:
SUBJECT = "Valentino"
CONDITION = "REC-0+2"
START_TIME = 172 # pick a 1 second window that has SPW-Rs

In [243]:
chans = channel_groups.full[SUBJECT]
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")
(times, sig, fs) = load_timeseries(bin_path, chans, start_time=(START_TIME), end_time=(START_TIME + 1.0))

nChan: 385, nFileSamp: 18000000


In [244]:
# sig = mean_subtract(sig)

## 1D kCSD on whole probe

In [245]:
gdx = intersite_distance = 0.020
k = get_kcsd(
    sig, intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
)
est_csd = k.values('CSD')

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-09
max lambda 0.0043
min lambda 1e-09
max lambda 0.0043
l-curve (all lambda):  0.23
Best lambda and R =  5.451979028328146e-10 ,  0.23


In [246]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_x, est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

lo = 3.8
hi = 6.1
ax.axhline(hi, alpha=0.5, color='k', linestyle="--")
ax.axhline(lo, alpha=0.5, color='k', linestyle="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [247]:
hpc_chans = chans[np.logical_and(k.ele_pos >= lo, k.ele_pos <= hi).flatten()] # These are the channel numbers belonging to the hippocampus
print(hpc_chans)

[380 383   1   2   5   6   9  10  13  14  17  18  21  22  25  26  29  30
  33  34  37  38  41  42  45  46  49  50  53  54  57  58  61  62  65  66
  69  70  73  74  77  78  81  82  85  86  89  90  93  94  97  98 101 102
 105 106 109 110 113 114 117 118 121 122 125 126 129 130 133 134 137 138
 141 142 145 146 149 150 153 154 157 158 161 162 165 166 169 170 173 174
 177 178 181 182 185 186 189 190 193 194 197 198 201 202 205 206 209 210
 213 214 217 218 221 222 225]


In [248]:
hpc_idx = np.where(np.logical_and(k.ele_pos >= lo, k.ele_pos <= hi).flatten())[0] # These are the indices of the above channels

In [249]:
assert np.array_equal(hpc_chans, chans[hpc_idx]) # Confirm that indexing is correct

In [250]:
print(f"Min HPC index: {hpc_idx.min()} \nMax HPC index: {hpc_idx.max() + 1}")

Min HPC index: 190 
Max HPC index: 305


## 1D kCSD on hippocampal channels

In [251]:
hpc_k = get_kcsd(
    sig[:, hpc_idx], intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
)
hpc_est_csd = hpc_k.values('CSD')

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0119
min lambda 1e-12
max lambda 0.0119
l-curve (all lambda):  0.23
Best lambda and R =  2.8175859627735274e-05 ,  0.23


In [252]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, hpc_k.estm_x, hpc_est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

# Visualize stratum radiatum position
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_mask = np.isin(hpc_chans, sr_chans)
sr_pos = hpc_k.ele_pos[sr_mask]
ax.axhline(sr_pos.min(), alpha=0.5, color='k', linestyle=":")
ax.axhline(sr_pos.max(), alpha=0.5, color='k', linestyle=":")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …